오차 역전파를 통해 오차를 기반으로 가중치를 수정한 후 더 좋은 성능을 낼 수 있도록 모형을 개선한다.

임의의 입력 값과 출력값의 설정

In [ ]:
import numpy as np

In [ ]:
input = np.array([[1,2,3]])
input.shape

(1, 3)

In [ ]:
target = np.array([[0.5,0.1]])
target.shape

(1, 2)

In [ ]:
print(input.shape, target.shape)

(1, 3) (1, 2)


임의의 가중치 설정

In [ ]:
w1 = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])
w1.shape

(2, 3)

In [ ]:
w2 = np.array([[0.7,0.9],[0.8,0.1]])
w2.shape

(2, 2)

In [ ]:
b1 = np.array([[0.5],[0.5]])
b1.shape

(2, 1)

In [ ]:
b2 = np.array([[0.3],[0.3]])
b2.shape

(2, 1)

순전파를 통한 출력값 계산

In [ ]:
print(input.shape,w1.shape)

(1, 3) (2, 3)


In [ ]:
g11 = w1@(input.T)+b1 # 순전파 계산
g11

array([[2.7],
       [3.3]])

In [ ]:
g12 = w1.dot(input.T)+b1
g12

array([[2.7],
       [3.3]])

In [ ]:
print(np.equal(g11, g12))

[[ True]
 [ True]]


In [ ]:
h = 1/(1+np.exp(-g11)) # 활성화 함수, 시그모이드 계산
print(h, h.shape)

[[0.93702664]
 [0.96442881]] (2, 1)


함수화

In [ ]:
def hidden_cal(x,w,b):
  g = w.dot(x)+b
  h = 1/(1+np.exp(-g))
  return h

In [ ]:
h2 = hidden_cal(h,w2,b2)
h2

array([[0.86103399],
       [0.75879129]])

비용 함수 정의 및 1차 미분식 계산

비용 함수로는 오차 제곱합을 사용, 미분식의 계산을 편리하게 하기 위한 1/2 연산의 추가

활성화 함수로 사용하는 시그모이드 함수의 미분

In [ ]:
from sympy import Derivative, symbols

In [ ]:
y1 = symbols('y1')
y2 = symbols('y2')

t1 = symbols('t1')
t2 = symbols('t2')

C = 1/2*(((y1-t1)**2)+((y2-t2)**2))

In [ ]:
fprime = Derivative(C, t1).doit()
fprime

1.0*t1 - 1.0*y1

In [ ]:
fprime = Derivative(C, t2).doit()
fprime

1.0*t2 - 1.0*y2

In [ ]:
x = symbols('x')

In [ ]:
fx = 1/(1+np.e**-x)

In [ ]:
fprime = Derivative(fx, x).doit()
fprime

1.0*2.71828182845905**(-x)/(1 + 2.71828182845905**(-x))**2

역전파를 통한 1차 미분값 구하기

은닉층에서 출력층까지 연결되는 가중치가 변했을 때 비용 함수의 변화량

In [ ]:
target.astype

<function ndarray.astype>

In [ ]:
target[0,0]

0.5

In [ ]:
target[0,1]

0.1

In [ ]:
h2

array([[0.86103399],
       [0.75879129]])

In [ ]:
h2[0]

array([0.86103399])

In [ ]:
h2[1]

array([0.75879129])

In [ ]:
h2[0]-target[0,0]

array([0.36103399])

In [ ]:
h2[0]*(1-h2[0])

array([0.11965446])

거꾸러 올라가는 과정의 수식 계산

In [ ]:
w2 # 은닉층의 가중치 행렬의 전치

array([[0.7, 0.9],
       [0.8, 0.1]])

In [ ]:
h2 # 은닉층의 입력값

array([[0.86103399],
       [0.75879129]])

In [ ]:
b2 # 은닉층의 편향 가중치

array([[0.3],
       [0.3]])

In [ ]:
print(w2.shape, h2.shape)

(2, 2) (2, 1)


In [ ]:
w2@(h2)+b2

array([[1.58563595],
       [1.06470632]])

In [ ]:
print(w2[0,0], h2[0,0])

0.7 0.8610339864424298


In [ ]:
w2[0,0]*h2[0,0]

0.6027237905097008

In [ ]:
w2[0,1]*h2[1,0]

0.6829121630087827

In [ ]:
w2[0,0]*h2[0,0]+w2[0,1]*h2[1,0]+b2[0]

array([1.58563595])

위 수식을 미분한 값을 얻기 위한 함수 작성

행렬곱을 펼친 수식을 얻어야 한다.


In [ ]:
x = symbols('x')
y = symbols('y')

b = x+y
c = b+y
c

x + 2*y

In [ ]:
def propagation_diff(w, h, b):
  """
  행렬 연산을 펼친 수식(문자열)을 얻기 위한 함수
  Arguments:
    w : 가중치 행렬의 전치
    h : 입력 데이터
    b : 가중치 편향값
  Return:
    list_num : 수식 문자열
  """
  num_unit = w.shape[1] # 은닉층의 unit 개수
  num_output = w.shape[0] # 출력층의 unit 개수, bias의 개수

  # 각 행렬의 기호 변수화
  for i in range(num_unit):
    exec(f"b{i} = symbols('b{i}')")
    exec(f"h{i}{0} = symbols('h{i}{0}')")
    for j in range(num_output):
      exec(f"w{i}{j}=symbols('w{i}{j}')")
  
  # 각 기호 변수들에 대한 미분 시행
  list_num = []
  list_mul = 0

  for i in range(num_unit):
    # 출력층의 i번째 입력 수식값 생성
    list_mul = eval(f"b{i}")    
    for j in range(num_output):
      list_mul += eval(f"w{i}{j}*h{j}{0}")

    list_num.append(list_mul)
    z1 = 0

  return list_num

In [ ]:
list_mul = propagation_diff(w2, h2, b2)
list_mul

[b0 + h00*w00 + h10*w01, b1 + h00*w10 + h10*w11]

In [ ]:
# 출력층에서 은닉층으로 거꾸로 거슬러 올라가는 과정
def propagation_matrix_element(matrix_list, num_unit, num_output):
  """
  Arguments:
    matrix_list : 
    num_unit : 
    numt_output :
  """
  # 각 행렬의 기호 변수화
  for i in range(num_unit):
    exec(f"b{i} = symbols('b{i}')")
    exec(f"h{i}{0} = symbols('h{i}{0}')")
    for j in range(num_output):
      exec(f"w{i}{j}=symbols('w{i}{j}')")

  fprime = []

  for i in range(num_unit):
    for j in range(num_output):
      fprime.append(eval(f"Derivative({matrix_list}[{i}], 'w{i}{j}').doit()"))
    fprime.append(eval(f"Derivative({matrix_list}[{i}], 'b{i}{0}').doit()"))

  return fprime

In [ ]:
list = propagation_matrix_element(list_mul, w2.shape[0], w2.shape[1])
for i in list:
  print(i)

h00
h10
0
h00
h10
0


출력값의 변화에 따른 오차의 변화량

z값의 변화에 따른 출력값의 변화량

한 가중치 변화에 따른 z값의  변화량

의 구현, 

In [ ]:
# 총 가중치의 개수, 가중치 행렬의 요소 개수 + 편향 개수 1
weight_num = w2.shape[0]*w2.shape[1]+1
weight_num

5

In [ ]:
# 각 가중치 변화에 따른 비용 함수의 변화량 계산
def propagation_matrix_element2(matrix_list, num_unit, num_output, predict, target, input):
  """
  Arguments:
    matrix_list : 
    num_unit : 
    numt_output :
    predict
    target
  """
  # 각 행렬의 기호 변수화
  for i in range(num_unit):
    exec(f"b{i} = symbols('b{i}')")
    exec(f"h{i}{0} = symbols('h{i}{0}')")
    for j in range(num_output):
      exec(f"w{i}{j}=symbols('w{i}{j}')")

  fprime = []

  for i in range(num_unit):
    for j in range(num_output):
      fprime.append(eval(f"Derivative({matrix_list}[{i}], 'w{i}{j}').doit()"))
    fprime.append(eval(f"Derivative({matrix_list}[{i}], 'b{i}{0}').doit()"))

  cal_list = []
  b = 0

  # 각 가중치에 대한 비용 함수의 변화량
  for i in range(num_unit):
    for j in range(num_output):
      a = eval(f"(predict[{i}] - target[0, {i}]) * (predict[{i}]*(1-predict[{i}]) * h[{j},0])")
      print(a)
    
    b += eval(f"(predict[{i}] - target[0, {i}]) * (predict[{i}]*(1-predict[{i}]))")
    
  print(b)

  return fprime

In [ ]:
propagation_matrix_element2(list_mul, w2.shape[0], w2.shape[1], h2, target, h)

[0.04047892]
[0.04166268]
[0.11298352]
[0.11628758]
[0.16377596]


[h00, h10, 0, h00, h10, 0]

In [ ]:
print(h2[0], h2[1,0], h[0,0], h[1,0])

[0.86103399] 0.7587912922319807 0.9370266439430035 0.9644288107273639


In [ ]:
list_mul

[b0 + h00*w00 + h10*w01, b1 + h00*w10 + h10*w11]

In [ ]:
list_mul[0]

b0 + h00*w00 + h10*w01

In [ ]:
list_mul[1]

In [ ]:
fprime = Derivative(list_mul[0], 'w00').doit()
fprime

In [ ]:
bw41 = (h2[0]-target[0,0]) * (h2[0]*(1-h2[0])) * h[0]
bw41

In [ ]:
bw42 = (h2[1]-target[0,1]) * (h2[1]*(1-h2[1])) * h[0]
bw42

In [ ]:
bw51 = (h2[0]-target[0,0]) * (h2[0]*(1-h2[0])) * h[1]
bw51

In [ ]:
bw52 = (h2[1]-target[0,1]) * (h2[1]*(1-h2[1])) * h[1]
bw52

In [ ]:
bb2 = (h2[0]-target[0,0]) * (h2[0]*(1-h2[0])) * 1 + (h2[1]-target[0,1]) * (h2[1]*(1-h2[1])) * 1
bb2

은닉층에서 입력층으로 거슬러 올라가는 계산

함수화와 확인

In [1]:
import numpy as np

In [16]:
input = np.array([[1],[2],[3]])
input.shape

(3, 1)

In [17]:
target = np.array([[0.5],[0.1]])
target.shape

(2, 1)

In [18]:
w1 = np.array([[0.1,0.2],[0.3,0.4],[0.5,0.6]])
w1.shape

(3, 2)

In [19]:
w2 = np.array([[0.7,0.8],[0.9,0.1]])
w2.shape

(2, 2)

In [20]:
b1 = np.array([[0.5],[0.5]])
b1.shape

(2, 1)

In [21]:
b2 = np.array([[0.3],[0.3]])
b2.shape

(2, 1)

In [23]:
def cal_dense(input, w, b):
  input_result = (w.T)@input+b
  hidden_output = 1/(1+np.exp(-input_result)) # 활성화 함수의 종류를 입력으로 받을 수 있음
  return hidden_output

In [24]:
hidden_output = cal_dense(input, w1, b1)
hidden_output

array([[0.93702664],
       [0.96442881]])

In [25]:
predict = cal_dense(hidden_output, w2, b2)
predict

array([[0.86103399],
       [0.75879129]])

In [28]:
c_diff = predict-target
c_diff

array([[0.36103399],
       [0.65879129]])

In [29]:
def w_c_diff(c_diff, output_result, input_result):
  """
  Args:
    c_diff : 비용 함수의 미분식
    output_result : 해당 층의 출력
    input_result : 이전 층의 출력
  Result:
    가중치들에 대한 비용 함수의 변화량, 편향에 대한 비용 함수의 변화량
  """
  d_c_matrix = []
  a = (c_diff*(output_result*(1-output_result)))@(input_result.T) # 각 가중치 변화량에 따른 비용 함수 변화량의 행렬 생성
  b = (c_diff.T)@(output_result*(1-output_result)) # 편향 가중치 변화에 따른 비용 함수의 변화량 
  return a, b

In [30]:
w_c_diff2, b_c_diff2 = w_c_diff(c_diff, predict, hidden_output)
w_c_diff2

array([[0.04047892, 0.04166268],
       [0.11298352, 0.11628758]])

In [31]:
b_c_diff2

array([[0.16377596]])

In [177]:
def w1_c_diff(c_diff, output_result, weight):
  a = weight@(c_diff*(output_result*(1-output_result))) # 은닉층에 대한 연산

  b = (c_diff*(output_result*(1-output_result)))*weight
  return a, b

In [178]:
h1_c_diff, b_c_diff1 = w1_c_diff(c_diff, predict, w2)
h1_c_diff

array([[0.12670084],
       [0.05093706]])

In [179]:
b_c_diff1

array([[0.03023953, 0.03455946],
       [0.10851897, 0.01205766]])

In [208]:
def input_c_diff(h_c_diff, hidden_output, input, weight, b, b_diff):
  a = h_c_diff*(hidden_output*(1-hidden_output))@(input.T) # 은닉층에 대한 연산
  
  b_c = b_diff*(hidden_output*(1-hidden_output)) * b
  return a, b_c

In [209]:
input_c_diff, b2_c_diff = input_c_diff(h1_c_diff, hidden_output,input, w1, b1, b_c_diff1)
input_c_diff

array([[0.00747633, 0.01495265, 0.02242898],
       [0.00174744, 0.00349488, 0.00524232]])

In [212]:
b2_c_diff

0.000892182711799219

In [214]:
sum = 0
for i in range(b2_c_diff.shape[0]):
  sum += b2_c_diff[i,i]

sum

0.0010990070941353347